# SparkContext and RDD basics



Spark revolves around the concept of a resilient distributed dataset (RDD), which is a fault-tolerant collection of elements that can be operated on in parallel. There are two ways to create RDDs: parallelizing an existing collection in your driver program, or referencing a dataset in an external storage system, such as a shared filesystem, HDFS, HBase, or any data source offering a Hadoop InputFormat.

### Import libraries

In [3]:
from pyspark import SparkContext
import numpy as np

## Initialize a `SparkContext` (the main abstraction to the cluster)
**Note the '4' in the argument. It denotes 4 cores to be used for this SparkContext object.**

In [4]:
# sc
sc=SparkContext(master="local[4]")

22/10/09 09:38:42 WARN Utils: Your hostname, Inspiron-5567 resolves to a loopback address: 127.0.1.1; using 192.168.1.72 instead (on interface wlp2s0)
22/10/09 09:38:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/09 09:38:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/09 09:38:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/09 09:38:43 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/09 09:38:43 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [5]:
print(sc)

<SparkContext master=local[4] appName=pyspark-shell>


In [6]:
sc=SparkContext(master="local[4]")

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[4]) created by __init__ at /tmp/ipykernel_9410/1749374336.py:2 

### Generate a list of random integeres

In [7]:
lst=np.random.randint(0,10,20)

In [8]:
print(lst)

[5 0 3 5 8 4 5 6 7 8 3 1 4 5 6 9 6 3 3 2]


### Parallelize the list - this is the main operation toward distributed computing

### What did we just do? We created a RDD? What is a RDD?
![](https://i.stack.imgur.com/cwrMN.png)
Spark revolves around the concept of a resilient distributed dataset (RDD), which is a **fault-tolerant collection of elements that can be operated on in parallel**. SparkContext manages the distributed data over the worker nodes through the cluster manager. 

There are two ways to create RDDs: 
* parallelizing an existing collection in your driver program, or 
* referencing a dataset in an external storage system, such as a shared filesystem, HDFS, HBase, or any data source offering a Hadoop InputFormat.

We created a RDD using the former approach

In [9]:
A=sc.parallelize(lst)

### `A` is a pyspark RDD object, we cannot access the elements directly

In [10]:
type(A)

pyspark.rdd.RDD

In [11]:
A

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

### Opposite to parallelization - `collect` brings all the distributed elements and returns them to the head node. <br><br>Note - this is a slow process, do not use it often. 

In [12]:
A.collect()

[5, 0, 3, 5, 8, 4, 5, 6, 7, 8, 3, 1, 4, 5, 6, 9, 6, 3, 3, 2]

### How were the partitions created? Use `glom` method

In [13]:
A.glom().collect()

[[5, 0, 3, 5, 8], [4, 5, 6, 7, 8], [3, 1, 4, 5, 6], [9, 6, 3, 3, 2]]

### Now stop the SC and reinitialize it with 2 cores and see what happens when you repeat the process!

In [ ]:
sc.stop()

In [ ]:
sc=SparkContext(master="local[2]")

In [ ]:
A = sc.parallelize(lst)

In [ ]:
A.glom().collect()

**The RDD is now distributed over two chunks, not four!** 

So, let's redo the process with 4 cores again.

In [ ]:
sc.stop()

In [ ]:
sc = SparkContext(master="local[4]")

In [ ]:
A = sc.parallelize(lst)

## Basic operations
### `Count` the elements

In [14]:
A.count()

20

### The first element (`first`) and the first few elements (`take`)

In [15]:
A.first()

5

In [16]:
A.take(4)

[5, 0, 3, 5]

### Removing duplicates: Get another RDD with only the `distinct` elements

The method `RDD.distinct()` Returns a new dataset that contains the distinct elements of the source dataset.

**NOTE**: This operation requires a **shuffle** in order to detect duplication across partitions. **So, it is a slow operation.**

In [17]:
A_distinct=A.distinct()

In [18]:
A_distinct.collect()

[0, 8, 4, 5, 1, 9, 6, 2, 3, 7]

### To sum all the elements use `reduce` method

In [ ]:
A.reduce(lambda x,y:x+y)

### Or direct `sum` method

In [ ]:
A.sum()

### Or using the `fold` method, which aggregates the elements of each partition, and then the results for all the partitions

In [20]:
A.fold(0,lambda x,y:x+y)

93

### Finding maximum element by `reduce`

In [22]:
A.reduce(lambda x,y: x if x > y else y)

9

### Finding longest word using `reduce`

In [ ]:
words = 'These are some of the best Macintosh computers ever'.split(' ')
wordRDD = sc.parallelize(words)
wordRDD.reduce(lambda w,v: w if len(w)>len(v) else v)

## Functions/filtering over RDD
### Use `filter` to return a new RDD with elements satisfying a given predicate (lambda expression)

In [ ]:
# Return RDD with elements divisible by 3
A.filter(lambda x:x%3==0).collect()

### Lambda functions are short and sweet but we can write regular Python functions to use with `reduce`

In [ ]:
def largerThan(x,y):
    """
    Returns the last word among the longest words in a list
    """
    if len(x)> len(y):
        return x
    elif len(y) > len(x):
        return y
    else:
        if x < y: return x
        else: return y

In [ ]:
wordRDD.reduce(largerThan)

## Sampling an RDD
* RDDs are often very large.
* **Aggregates, such as averages, can be approximated efficiently by using a sample.** This comes handy often for operation with extremely large datasets where a sample can tell a lot about the pattern and descriptive statistics of the data.
* Sampling is done in parallel and requires limited computation.

The method `RDD.sample(withReplacement,p)` generates a sample of the elements of the RDD. where
- `withReplacement` is a boolean flag indicating whether or not a an element in the RDD can be sampled more than once.
- `p` is the probability of accepting each element into the sample. Note that as the sampling is performed independently in each partition, the number of elements in the sample changes from sample to sample.

In [ ]:
# get a sample whose expected size is m
# Note that the size of the sample is different in different runs
m=5
n=20
print('sample1=',A.sample(False,m/n).collect()) 
print('sample2=',A.sample(False,m/n).collect())
print('sample3=',A.sample(False,m/n).collect())
print('sample4=',A.sample(False,m/n).collect())

### Things to note and think about
* Each time you run the previous cell, you get a different estimate
* The accuracy of the estimate is determined by the size of the sample $n*p$. Here, probability $p=\frac{m}{n}$
* See how the error changes as you vary $p$

## Basic statistics

In [ ]:
print("Maximum: ",A.max())
print("Minimum: ",A.min())
print("Mean (average): ",A.mean())
print("Standard deviation: ",A.stdev())

In [ ]:
A.stats()

## Mapping
### `map` operation with _lambda_ function

In [ ]:
B=A.map(lambda x:x*x)

In [ ]:
B.collect()

### `map` operation with regular Python function

In [ ]:
def square_if_odd(x):
    if x%2==1:
        return x*x
    else:
        return x

In [ ]:
A.map(square_if_odd).collect()

### `flatmap` method returns a new RDD by first applying a function to all elements of this RDD, and then flattening the results

In [ ]:
A.flatMap(lambda x:(x,x*x)).collect()

## Grouping and binning
### `groupby` returns a RDD of grouped elements (iterable) as per a given group operation (function)

In [ ]:
result=A.groupBy(lambda x:x%2).collect()
print(A.collect())
#print(sorted(result[0][1]))
sorted([(x, sorted(y)) for (x, y) in result])

### `histogram` method takes a list of bins/buckets and returns a tuple with result of the histogram (binning) 

In [ ]:
A.histogram([x for x in range(0,100,10)])

## Set operations
### Create smaller RDDs to demonstrate joint operations

In [ ]:
lst1=np.random.randint(0,10,3)
C=sc.parallelize(lst1)
lst2=np.random.randint(10,20,3)
D=sc.parallelize(lst2)
print("C:",C.collect())
print("D:",D.collect())

### `C+D` gives the union (like set union), not the element wise sum

In [ ]:
(C+D).collect()

### `cartesian` gives the pairwise product (as tuples) 

In [ ]:
C.cartesian(D).collect()

### `intersection` and `subtract `methods return a RDD of the set intersection and subtraction (difference)

In [ ]:
rdd1 = sc.parallelize([1, 10, 2, 3, 4, 5])
rdd2 = sc.parallelize([1, 6, 2, 3, 7, 8])
rdd1.intersection(rdd2).collect()

In [ ]:
rdd1.subtract(rdd2).collect()

### Stop the `SparkContext` at the end

In [ ]:
sc.stop()